# 🚀 Agent Framework to Foundry Agents: A Step-by-Step Walkthrough

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **© 2025 Ozgur Guler. All rights reserved.**

---

This notebook guides you through the progression of building AI agents, from simple local agents to Azure AI Foundry managed agents.

## What You'll Learn

1. **Simple Agent Framework Agent** — Run an agent locally that calls Azure OpenAI
2. **Agent Framework with Reasoning** — Add reasoning capabilities (o1/o3 models)
3. **Foundry Agent Service (Classic)** — Create agents managed by Azure AI Foundry
4. **Agent Framework + Foundry** — Best of both worlds: your code + Foundry's managed runtime

## Prerequisites

- Python 3.10+
- Azure subscription with Azure OpenAI access
- `.env` file configured (see `00-environment-setup/`)

## 📦 Setup: Install Dependencies & Load Environment

In [ ]:
%pip install -q python-dotenv azure-identity azure-ai-agents
%pip install -q agent-framework-core agent-framework-azure-ai

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

required_vars = ["AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME", "AZURE_OPENAI_API_VERSION"]
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    print(f"⚠️ Missing: {missing}")
else:
    print("✅ Environment loaded!")
    print(f"   Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')[:50]}...")

---

## 1️⃣ Simple Agent Framework Agent (`01-af-standard-agent.py`)

### What This Does

The **simplest possible agent**:
- Runs **locally on your machine**
- Calls Azure OpenAI's **Responses API** over HTTPS
- Uses **Agent Framework** for a clean API

### Architecture

```
┌─────────────────────┐         HTTPS          ┌─────────────────────┐
│   Your Computer     │ ───────────────────▶   │  Azure OpenAI       │
│   (Python script)   │ ◀───────────────────   │  (Model endpoint)   │
└─────────────────────┘                        └─────────────────────┘
```

### Key Concepts

| Concept | Explanation |
|---------|-------------|
| `AzureOpenAIResponsesClient` | Client that wraps Azure OpenAI Responses API |
| `create_agent()` | Creates an agent with name and instructions |
| `agent.run()` | Sends a message and gets a response |

In [ ]:
# 01-af-standard-agent.py — Simple Agent Framework Agent
# Agent runs LOCALLY, calls Azure OpenAI over HTTPS

import os
import asyncio
from agent_framework.azure import AzureOpenAIResponsesClient

async def run_simple_agent():
    # Step 1: Create client that connects to Azure OpenAI
    client = AzureOpenAIResponsesClient(
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    )
    
    # Step 2: Create an "agent" with a persona (local Python object)
    agent = client.create_agent(
        name="physicsbot",
        instructions="You are a professor in astrophysics. Be concise.",
    )
    
    # Step 3: Run the agent (calls Azure OpenAI)
    response = await agent.run("Are we in a black hole?")
    return response

result = await run_simple_agent()
print("🤖 Agent Response:")
print(result)

### Why Start Here?

This is the **baseline**:
- ✅ Verify Azure OpenAI connection works
- ✅ Understand the basic pattern (create → configure → run)
- ✅ See that an "agent" is just code that calls an API

---

## 2️⃣ Agent with Reasoning (`02-af-standard-agent-reasoning.py`)

### What's Different?

Uses a **reasoning model** (o1/o3) that "thinks" before responding.

| Standard Agent | Reasoning Agent |
|---------------|----------------|
| GPT-4o | o1/o3 models |
| Direct response | Thinks first |
| Fast | Slower but more accurate |

### Key Addition: `reasoning_effort`

```python
reasoning_effort="minimal"  # or "medium" or "high"
```

In [ ]:
# 02-af-standard-agent-reasoning.py — Agent with Reasoning Model

import os
from agent_framework.azure import AzureOpenAIResponsesClient

async def run_reasoning_agent():
    reasoning_deployment = os.environ.get("AZURE_OPENAI_REASONING_DEPLOYMENT_NAME")
    if not reasoning_deployment:
        print("⚠️ No reasoning model configured, using standard deployment")
        reasoning_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
    
    client = AzureOpenAIResponsesClient(
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        deployment_name=reasoning_deployment,
        api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    )
    
    # NEW: reasoning_effort parameter (only works with o1/o3 models)
    agent = client.create_agent(
        name="physicsbot",
        instructions="You are a professor in astrophysics.",
        reasoning_effort="minimal",
    )
    
    return await agent.run("Are we in a black hole? Explain your reasoning.")

try:
    result = await run_reasoning_agent()
    print("🧠 Reasoning Agent:")
    print(result)
except Exception as e:
    print(f"⚠️ Error: {e}")

---

## 3️⃣ Foundry Agent Service — Classic (`03-foundry-agent-basic.py`)

### 🔄 This is a Fundamental Shift!

Instead of running the agent locally:
- The **agent lives in Azure AI Foundry**
- The **thread (conversation) lives in Foundry**
- The **run (execution) happens in Foundry**

Your code becomes a **client/orchestrator**.

### Architecture Comparison

```
LOCAL (Steps 1-2):                       FOUNDRY (Step 3):
┌──────────────────────┐                 ┌──────────────────────┐
│  Your Code           │                 │  Your Code           │
│  ┌────────────────┐  │                 │  (orchestrator only) │
│  │ Agent Logic    │  │                 └──────────┬───────────┘
│  │ Thread/History │  │                            │
│  └────────────────┘  │                            ▼
└──────────┬───────────┘                 ┌──────────────────────┐
           ▼                             │  Azure AI Foundry    │
┌──────────────────────┐                 │  ┌────────────────┐  │
│  Azure OpenAI        │                 │  │ Agent + Thread │  │
│  (Model only)        │                 │  │ + Execution    │  │
└──────────────────────┘                 │  └────────────────┘  │
                                         └──────────────────────┘
```

### Key Concepts

| Concept | Explanation |
|---------|-------------|
| `AgentsClient` | Client to talk to Foundry Agent Service |
| `create_agent()` | Creates agent **in Foundry** (returns ID) |
| `threads.create()` | Creates conversation **in Foundry** |
| `runs.create_and_process()` | Executes agent **in Foundry** |

In [ ]:
# 03-foundry-agent-basic.py — Foundry Agent Service (Classic)
# Agent runs IN FOUNDRY, not locally!

import os
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool

def run_foundry_agent():
    # Foundry PROJECT endpoint (not Azure OpenAI!)
    # Format: https://<name>.services.ai.azure.com/api/projects/<project>
    project_endpoint = os.getenv("PROJECT_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    
    if not project_endpoint:
        raise ValueError("Missing PROJECT_ENDPOINT")
    
    print(f"📍 Foundry: {project_endpoint[:60]}...")
    
    credential = DefaultAzureCredential()
    agents_client = AgentsClient(endpoint=project_endpoint, credential=credential)
    
    with agents_client:
        code_interpreter = CodeInterpreterTool()
        
        # Step 1: Create agent IN FOUNDRY
        agent = agents_client.create_agent(
            model=model_deployment,
            name="Quickstart",
            instructions="Be concise.",
            tools=code_interpreter.definitions,
            tool_resources=code_interpreter.resources,
        )
        print(f"✅ Created agent: {agent.id}")
        
        # Step 2: Create thread IN FOUNDRY
        thread = agents_client.threads.create()
        print(f"✅ Created thread: {thread.id}")
        
        # Step 3: Add message IN FOUNDRY
        agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content="Write a haiku about Azure AI Foundry.",
        )
        
        # Step 4: Run agent IN FOUNDRY
        run = agents_client.runs.create_and_process(
            thread_id=thread.id,
            agent_id=agent.id,
        )
        print(f"✅ Run status: {run.status}")
        
        # Step 5: Fetch conversation FROM FOUNDRY
        messages = agents_client.messages.list(thread_id=thread.id)
        print("\n📝 Conversation:")
        for m in messages:
            content = m.content[0].text.value if m.content else str(m.content)
            print(f"{'🧑' if m.role == 'user' else '🤖'} {m.role}: {content}")

try:
    run_foundry_agent()
except Exception as e:
    print(f"⚠️ Error: {e}")
    print("Make sure PROJECT_ENDPOINT is your Foundry project endpoint.")

### Why Use Foundry Agent Service?

| Benefit | Explanation |
|---------|-------------|
| **Managed runtime** | No infrastructure to manage |
| **Built-in tools** | Code Interpreter, File Search |
| **Thread persistence** | Conversations stored in Azure |
| **Scalability** | Foundry handles scaling |

---

## 4️⃣ Agent Framework + Foundry (`03-af-foundry-agent-basic.py`)

### Best of Both Worlds

- **Agent Framework**: Clean, simple API
- **Foundry Agent Service**: Managed runtime

### Code Comparison

**Foundry Only (verbose):**
```python
agent = agents_client.create_agent(...)
thread = agents_client.threads.create()
agents_client.messages.create(thread_id=thread.id, ...)
run = agents_client.runs.create_and_process(...)
messages = agents_client.messages.list(...)
```

**Agent Framework + Foundry (clean):**
```python
async with AzureAIAgentClient(...).create_agent(...) as agent:
    result = await agent.run("Your message")
```

In [ ]:
# 03-af-foundry-agent-basic.py — Agent Framework + Foundry
# Clean API + Managed execution

import os
from azure.identity.aio import DefaultAzureCredential
from azure.ai.agents.aio import AgentsClient
from agent_framework.azure import AzureAIAgentClient

async def run_af_foundry_agent():
    project_endpoint = os.getenv("PROJECT_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    
    if not project_endpoint:
        raise ValueError("Missing PROJECT_ENDPOINT")
    
    print(f"📍 Foundry: {project_endpoint[:60]}...")
    
    async with DefaultAzureCredential() as credential:
        async with AgentsClient(endpoint=project_endpoint, credential=credential) as agents_client:
            
            # AzureAIAgentClient wraps Foundry with Agent Framework API
            # should_cleanup_agent=True: delete agent when done
            async with AzureAIAgentClient(
                agents_client=agents_client,
                model_deployment_name=model_deployment,
                should_cleanup_agent=True,
            ).create_agent(
                name="Quickstart",
                instructions="Be concise.",
            ) as agent:
                
                # Looks like local Agent Framework, executes in Foundry!
                result = await agent.run("Write a haiku about Azure AI Foundry.")
                print("🤖 Response:")
                print(result)

try:
    await run_af_foundry_agent()
except Exception as e:
    print(f"⚠️ Error: {e}")

---

## 📊 Summary: Choosing the Right Approach

| Approach | File | Best For | Trade-offs |
|----------|------|----------|------------|
| **Local Agent Framework** | `01-af-standard-agent.py` | Prototyping, full control | You manage everything |
| **Local + Reasoning** | `02-af-standard-agent-reasoning.py` | Complex reasoning tasks | Slower, requires o1/o3 |
| **Foundry Classic** | `03-foundry-agent-basic.py` | Zero-ops, built-in tools | Verbose API |
| **AF + Foundry** | `03-af-foundry-agent-basic.py` | Production with clean code | Best of both |

## 🎯 What's Next?

**`02-azd-deploy-hosted-agent/`** — Containerized agents

Hosted agents give you:
- Your code in a container (full dependency control)
- Foundry as the control plane (deploy/version/rollback)
- Enterprise governance (Agent ID, Conditional Access)
- MCP tool support via APIM

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>